In [7]:
import json, os, re, requests, sys, time
import pandas as pd
import logging
import logging.handlers

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime
from pykrx import stock
from webob.compat import urlparse
import lxml
URL_NAVER_NEWS_QUERY = "https://search.naver.com/search.naver?where=news&query=%s&sm=tab_opt&pd=3&ds=%s&de=%s&start=%s"

In [8]:
class NaverNewsCrawler:
    def __init__(self):
        self.ticker = None
        self.result = []
        pass

    def _crawl_by_query(self, maxpage, ticker, s_date, e_date):

        q=""
        for c in ticker.encode("UTF-8"):
            q += "%%%s" % format(c, "x").upper()
        print(q)

        # 최대페이지(입력)까지 loop 돌면서 뉴스 가져오기(네이버뉴스가 없는 페이지가 있기때문)
        page = 1
        n_news = 0

        while True:
            if page <= maxpage:
                page_t = (lambda x: (x-1)*10+1)(page)

                r_url = URL_NAVER_NEWS_QUERY % (q, s_date, e_date, page_t,)
                print('url =',r_url)
                r = requests.get(r_url)
                soup = bs(r.text, 'lxml')
                news = soup.find('div', class_='group_news').find("ul",class_="list_news")

                for urls in news.find_all("a", {"class": "info"}):
                    urls.get('href')   
                    if urls['href'].startswith("https://news.naver.com"):
                        n_url = urls['href']
                        res = self._crawl_content(n_url)
                        self.result.append(res)
                        n_news += 1
                        time.sleep(1)

                page += 1
            else:
                break

        print("number of news for comp {} = {}".format(ticker, n_news))

      
    def _crawl_content(self, n_url):

        r = requests.get(n_url, headers={'User-Agent':'Mozilla/5.0'})
        soup = bs(r.text,'lxml')
        # 제목
        news_title = soup.select("h3#articleTitle")[0].text
        # 기사일자
        pdate = soup.find('span',class_='t11').text[:10]
        # news_detail.append(pdate)
        # 본문
        content = soup.select('#articleBodyContents')[0]
        cntnt = content.get_text().replace('\n', " ").strip('\t\r')
        # 기사링크
        url = n_url
        # 기사ID
        p = urlparse.parse_qs(urlparse.urlparse(url).query)
        article_id = p['mode'][0]+p['mid'][0]+p['sid1'][0]+p['oid'][0]+p['aid'][0]
        # 언론사
        # pcompany = soup.select('#footer address')[0].a.get_text()

        return {"title":news_title, "date":pdate, "contents":cntnt, "url":url}


    def get_news(self, maxpage, ticker, s_date, e_date):
        # logger instance 생성
        logger = logging.getLogger(__name__)
        
        # Check handler exists
        # if len(logger.handlers) > 0:
        #     return logger # Logger already exists
               
        # Format 설정
        formatter = logging.Formatter(
            "%(asctime)s %(levelname)s [%(name)s] [%(filename)s:%(lineno)d] - %(message)s"
        )
        
        # handler 생성
        streamHandler = logging.StreamHandler()
        streamHandler.setFormatter(formatter)
        
        logger.addHandler(streamHandler)
        logger.setLevel(level=logging.INFO)
        
        today = datetime.today().strftime("%Y.%m.%d")
        print(f'today= {today}')
        
        if s_date is None:
            s_date = today
        else:
            s_date = s_date

        if e_date is None:
            e_date = today
        else:
            e_date = e_date
                    
        for i, ticker in enumerate([ticker]):
            logger.info("i = {}, crypto = {}".format(i, ticker))
            try:
                msg = "crypto = {}".format(ticker)
                logger.info(msg)
                time.sleep(1)
                self._crawl_by_query(maxpage=maxpage, ticker=ticker, s_date=s_date, e_date=e_date)
            except Exception as e:
                logger.error(e)
                pass
        

# dd = '2022.03.01'

# datetime.strptime(dd, '%Y.%m.%d')
# datetime.strftime(datetime.strptime(dd, '%Y.%m.%d'), '%Y-%m-%d')

# from urllib import parse
 
 
# url = parse.urlparse('http://www.exeam.org?examParam1=value1&examParam2=이더리움') 
 
 
# query = parse.parse_qs(url.query)
# parse.urlencode(query, doseq=True) #examParam2=%ED%95%9C%EA%B8%80&examParam1=value1

In [9]:
NNC = NaverNewsCrawler()

In [10]:
NNC.get_news(2, '이더리움', '2022.03.10', '2022.03.15')

2022-04-01 16:05:42,243 INFO [__main__] [841769937.py:101] - i = 0, crypto = 이더리움
2022-04-01 16:05:42,243 INFO [__main__] [841769937.py:101] - i = 0, crypto = 이더리움
2022-04-01 16:05:42,246 INFO [__main__] [841769937.py:104] - crypto = 이더리움
2022-04-01 16:05:42,246 INFO [__main__] [841769937.py:104] - crypto = 이더리움


today= 2022.04.01
%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80
url = https://search.naver.com/search.naver?where=news&query=%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80&sm=tab_opt&pd=3&ds=2022.03.10&de=2022.03.15&start=1
url = https://search.naver.com/search.naver?where=news&query=%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80&sm=tab_opt&pd=3&ds=2022.03.10&de=2022.03.15&start=11
number of news for comp 이더리움 = 3


In [11]:
NNC.result

[{'title': "비트코인·이더리움 유럽 거래 금지되나…EU 새 규제안에 '발칵'",
  'date': '2022.03.14',
  'contents': '     \t \t[러시아-우크라 사태, 금리 인상 등으로 침체기…유럽의회 표결 여부에 \'촉각\'](서울=뉴스1) 장수영 기자 = 7일 오후 서울시 서초구 빗썸 고객지원센터 전광판에 비트코인 등 주요 가상자산(암호화폐) 시세가 나타나고 있다.   최근 러시아의 우크라이나 침공과 관련해 상승의 움직임을 보이던 비트코인 등 가상자산(암호화폐)의 가격이 국제 유가 급등 영향으로 일제히 하락하는 움직임이 나타나고 있다. 2022.3.7/뉴스1    최근 침체에서 벗어나지 못하고 있는 암호화폐(가상자산) 시장에 또 한번 악재가 닥쳤다. EU(유럽연합)이 암호화폐에 대해 새로운 규제안을 표결에 부칠 예정으로 사실상 유럽에서 비트코인과 이더리움을 포함한 주요 암호화폐의 사용을 금지시킬 가능성도 있기 때문이다. 13일(현지시간) 블룸버그에 따르면 EU의 입법부인 유럽의회는 가상자산 관련 법규 \'MiCA\'(Markets in Crypto-Assets)에 비트코인과 이더리움 등 주요 작업증명(Proof of Work) 방식의 가상자산 사용에 제재를 가하는 내용을 담고, 이를 표결에 부친다. 블룸버그에 따르면 이 법안의 최종 초안은 "EU에서 발행 및 거래되는 암호화폐 자산은 최소한의 환경 지속 가능성 기준의 적용을 받아야 하고 이러한 요건을 준수하기 위한 단계별 계획을 수립, 유지해야 한다"고 말했다. EU의 가상자산 규제안은 가장 자산 법제화를 통해 시장을 양성화하고 성장을 돕는다는 취지로 발의됐으며 오는 14일(현지시간) 경제통화위원회가 표결할 예정이다. 이 매체는 \'지속 가능성\'과 \'작업 증명\' 등을 언급한 점 등이 비트코인과 이더리움 등 암호화폐에 대한 사용 및 거래 금지를 위해 도입된 조항으로 해석했다. 작업증명 방식의 암호화폐는 채굴과 거래과정에서 환경 오염에 대한 논란이 끊임없이 제기되고 있다. 

In [12]:
df = pd.DataFrame(NNC.result)
df = df[['date','title','url','contents']]
df.head()

,date,title,url,contents
0,2022.03.14,비트코인·이더리움 유럽 거래 금지되나…EU 새 규제안에 '발칵',https://news.naver.com/main/read.naver?mode=LS...,"\t \t[러시아-우크라 사태, 금리 인상 등으로 침체기…유럽의회 표결 여..."
1,2022.03.15,"이더리움, 연말까지 100% 이상 상승한 770만원 돌파 가능",https://news.naver.com/main/read.naver?mode=LS...,◆…자료:유튜브이더리움에 대한 긍정적 전망이 우세한 가운데 올해 안에 시가...
2,2022.03.15,"EU, 가상화폐 관련 규제 검토… ""유럽서 비트코인·이더리움 금지될 수도"" [글로벌이슈]",https://news.naver.com/main/read.naver?mode=LS...,"동영상 뉴스 EU, 가상화폐 관련 규제 검토… ""유럽서 비트코인·이더..."
